In [1]:
import os
import sys
import json
import openai
from dotenv import load_dotenv

import psycopg
from psycopg.rows import dict_row

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate,PromptTemplate
from langchain.chains import ConversationChain, LLMChain
from langchain.schema.output_parser import StrOutputParser

from langchain.agents.agent_types import AgentType
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory,ConversationSummaryBufferMemory,ConversationSummaryMemory,ReadOnlySharedMemory
from langchain.schema import HumanMessage, AIMessage


from datetime import datetime
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage,message_to_dict,messages_from_dict




In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')


In [3]:
llms = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=OPENAI_API_KEY)

In [4]:
db_config = {
    'dbname': os.environ.get('db_name'),
    'dbuser': os.environ.get('db_username'),
    'dbpassword': os.environ.get('db_password'),
    'dbhost': os.environ.get('db_host'),
    'dbport': os.environ.get('db_port'),
}

In [5]:
from typing import List

class BaseDBStorage(BaseChatMessageHistory):
    def __init__(self, db_name:str, db_user:str, db_password:str, db_host:str, db_port:str, table_name:str, chat_id:str):
        self.db_config = {
            'dbname': db_name,     
            'user': db_user,        
            'password': db_password,    
            'host': db_host,           
            'port': db_port,
            'table_name': table_name,                 
        }
        
        self.chat_id = chat_id
        
        
        try:
            self.conn = psycopg.connect(dbname=self.db_config['dbname'], user=self.db_config['user'], password=self.db_config['password'], host=self.db_config['host'], port=self.db_config['port'])
            self.cur = self.conn.cursor(row_factory=dict_row)
            self.create_table_if_not_exists()
        except psycopg.Error as e:
            print("Error connecting to PostgreSQL database:", e)
            self.conn = None
            self.cur = None
                  
    
    def create_table_if_not_exists(self):
        create_table_query = f"""CREATE TABLE IF NOT EXISTS {self.db_config['table_name']} (
            id SERIAL PRIMARY KEY,
            chat_id TEXT NOT NULL,
            message JSONB NOT NULL, 
            time_stamp TIMESTAMP NOT NULL
        );"""
        self.cur.execute(create_table_query)
        self.conn.commit()
    
    def add_ai_message(self, message: str) -> None:
        return self.add_message(AIMessage(content=message))
    
    def add_user_message(self, message: str) -> None:
        return self.add_message(HumanMessage(content=message))
    
    @property
    def messages(self) -> List[BaseMessage]:  # type: ignore
        """Retrieve the messages from PostgreSQL"""
        query = (
            f"SELECT * FROM {self.db_config['table_name']} WHERE chat_id = %s ORDER BY time_stamp;"
        )
        self.cur.execute(query, (self.chat_id,))
        items = [record["message"] for record in self.cur.fetchall()]
        messages = messages_from_dict(items)
        return messages
    
    
    
    def add_message(self, message:BaseMessage):
        time_stamp = datetime.now()
        query = f"INSERT INTO {self.db_config['table_name']} (chat_id, message, time_stamp) VALUES (%s, %s, %s);"
        self.cur.execute(query, (self.chat_id, json.dumps(message_to_dict(message)), time_stamp))
        self.conn.commit()
        
            
    def clear(self) -> None:
        conn = psycopg.connect(self.db_config)
        cur = conn.cursor()
        if conn:
            try:
                query = f"DELETE FROM {self.db_config['dbname']} WHERE session_id = %s;"
                cur.execute(query, (self.session_id,))
                conn.commit()
                cur.close()
                conn.close()
            except psycopg.Error as e:
                print("Error clearing DB:", e)
        else:
            print("Failed to establish connection...")

In [6]:
chat_id = '485276c7-8d9b-48b1-a8aa-ad65a7118192'

In [7]:
base_storage = BaseDBStorage(db_host=os.environ.get('db_host'), db_name=os.environ.get('db_name'), db_user=os.environ.get('db_username'), db_password=os.environ.get('db_password'), db_port=os.environ.get('db_port'), table_name=os.environ.get('conversation_table_name'), chat_id=chat_id)


In [8]:
memory  = ConversationSummaryMemory.from_messages(chat_memory=base_storage, llm=llms, return_messages=True,verbose=True)
conversation_bufw = ConversationChain(
    llm=llms, 
    memory=memory
)


In [9]:
memory.chat_memory.messages

[HumanMessage(content='Hello, how are you?'),
 AIMessage(content='Hello, how are you?'),
 HumanMessage(content='Can you tell me about yourself?'),
 AIMessage(content='Of course! I am an AI language model developed by OpenAI called GPT-3. I have been trained on a wide range of data sources, including books, articles, and websites, to generate human-like responses to text inputs. I can assist with various tasks, answer questions, engage in conversations, and even provide creative writing suggestions. Is there anything specific you would like to know about me?'),
 HumanMessage(content='How AI can be used in healthcare?'),
 AIMessage(content='AI can be used in healthcare in a variety of ways. One of the main applications is in medical imaging, where AI algorithms can analyze images such as X-rays, CT scans, and MRIs to help detect and diagnose diseases. AI can also be used to predict patient outcomes and identify high-risk patients, allowing healthcare providers to intervene earlier and pr

In [10]:
memory.buffer

'The human greets the AI and asks how it is doing. The AI responds by returning the greeting and asking how the human is doing. The human then asks the AI to tell them about itself. The AI explains that it is an AI language model called GPT-3 developed by OpenAI. It has been trained on various data sources and can assist with tasks, answer questions, engage in conversations, and provide creative writing suggestions. The AI asks if there is anything specific the human would like to know about it. The human asks how AI can be used in healthcare. The AI explains that AI can be used in healthcare in a variety of ways, such as medical imaging analysis, predicting patient outcomes, drug discovery, and development, and providing patient support through chatbots and virtual assistants. AI has the potential to greatly improve healthcare outcomes and efficiency. Specifically, AI can be used in healthcare to analyze medical images and assist in diagnosing diseases, predict patient outcomes, aid i

In [11]:
memory.chat_memory.messages[-1].content

'There are several algorithms that can be used to detect abnormalities in analyzing medical images. Some of the commonly used algorithms include Convolutional Neural Networks (CNN), Support Vector Machines (SVM), Random Forests, Decision Trees, and Deep Learning algorithms. CNNs are particularly effective in image analysis tasks, as they can automatically learn and extract features from images. SVMs, on the other hand, are a type of supervised learning algorithm that can classify data into different categories based on the features extracted from the images. These algorithms, along with others, contribute to the accurate detection of abnormalities in medical images.'

In [12]:
memory.predict_new_summary(memory.chat_memory.messages, "") 

'The human asks the AI about the use of AI in healthcare. The AI explains that AI can be used in various ways in healthcare, such as analyzing medical images, predicting patient outcomes, assisting in drug discovery, and providing patient support. The human then asks specifically about the algorithms used to detect abnormalities in medical images. The AI responds that there are several algorithms that can be used, including Convolutional Neural Networks (CNN), Support Vector Machines (SVM), Random Forests, Decision Trees, and Deep Learning algorithms. These algorithms contribute to the accurate detection of abnormalities in medical images.'

In [13]:
memory.buffer

'The human greets the AI and asks how it is doing. The AI responds by returning the greeting and asking how the human is doing. The human then asks the AI to tell them about itself. The AI explains that it is an AI language model called GPT-3 developed by OpenAI. It has been trained on various data sources and can assist with tasks, answer questions, engage in conversations, and provide creative writing suggestions. The AI asks if there is anything specific the human would like to know about it. The human asks how AI can be used in healthcare. The AI explains that AI can be used in healthcare in a variety of ways, such as medical imaging analysis, predicting patient outcomes, drug discovery, and development, and providing patient support through chatbots and virtual assistants. AI has the potential to greatly improve healthcare outcomes and efficiency. Specifically, AI can be used in healthcare to analyze medical images and assist in diagnosing diseases, predict patient outcomes, aid i

In [14]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human greets the AI and asks how it is doing. The AI responds by returning the greeting and asking how the human is doing. The human then asks the AI to tell them about itself. The AI explains that it is an AI language model called GPT-3 developed by OpenAI. It has been trained on various data sources and can assist with tasks, answer questions, engage in conversations, and provide creative writing suggestions. The AI asks if there is anything specific the human would like to know about it. The human asks how AI can be used in healthcare. The AI explains that AI can be used in healthcare in a variety of ways, such as medical imaging analysis, predicting patient outcomes, drug discovery, and development, and providing patient support through chatbots and virtual assistants. AI has the potential to greatly improve healthcare outcomes and efficiency. Specifically, AI can be used in healthcare to analyze medical images and assist in diagnosing diseas

In [15]:
result = conversation_bufw.predict(input="Which Algorithm is will be used to detect abnormality in analyzing medical images")
print(result) 

There are several algorithms that can be used to detect abnormalities in analyzing medical images. Some of the commonly used algorithms include Convolutional Neural Networks (CNN), Support Vector Machines (SVM), Random Forests, Decision Trees, and Deep Learning algorithms. CNNs are particularly effective in image analysis tasks, as they can automatically learn and extract features from images. SVMs, on the other hand, are a type of supervised learning algorithm that can classify data into different categories based on the features extracted from the images. These algorithms, along with others, contribute to the accurate detection of abnormalities in medical images.


In [16]:
memory.chat_memory.messages

[HumanMessage(content='Hello, how are you?'),
 AIMessage(content='Hello, how are you?'),
 HumanMessage(content='Can you tell me about yourself?'),
 AIMessage(content='Of course! I am an AI language model developed by OpenAI called GPT-3. I have been trained on a wide range of data sources, including books, articles, and websites, to generate human-like responses to text inputs. I can assist with various tasks, answer questions, engage in conversations, and even provide creative writing suggestions. Is there anything specific you would like to know about me?'),
 HumanMessage(content='How AI can be used in healthcare?'),
 AIMessage(content='AI can be used in healthcare in a variety of ways. One of the main applications is in medical imaging, where AI algorithms can analyze images such as X-rays, CT scans, and MRIs to help detect and diagnose diseases. AI can also be used to predict patient outcomes and identify high-risk patients, allowing healthcare providers to intervene earlier and pr

In [17]:
memory.chat_memory.messages[-1].content

'There are several algorithms that can be used to detect abnormalities in analyzing medical images. Some of the commonly used algorithms include Convolutional Neural Networks (CNN), Support Vector Machines (SVM), Random Forests, Decision Trees, and Deep Learning algorithms. CNNs are particularly effective in image analysis tasks, as they can automatically learn and extract features from images. SVMs, on the other hand, are a type of supervised learning algorithm that can classify data into different categories based on the features extracted from the images. These algorithms, along with others, contribute to the accurate detection of abnormalities in medical images.'

In [18]:
memory.predict_new_summary(memory.chat_memory.messages, "") 

'The human asks the AI about the use of AI in healthcare. The AI explains that AI can be used in various ways in healthcare, such as analyzing medical images, predicting patient outcomes, assisting in drug discovery, and providing patient support. The human then asks specifically about the algorithms used to detect abnormalities in medical images. The AI responds that there are several algorithms that can be used, including Convolutional Neural Networks (CNN), Support Vector Machines (SVM), Random Forests, Decision Trees, and Deep Learning algorithms. These algorithms contribute to the accurate detection of abnormalities in medical images.'

In [19]:
memory.buffer

'The human greets the AI and asks how it is doing. The AI responds by returning the greeting and asking how the human is doing. The human then asks the AI to tell them about itself. The AI explains that it is an AI language model called GPT-3 developed by OpenAI. It has been trained on various data sources and can assist with tasks, answer questions, engage in conversations, and provide creative writing suggestions. The AI asks if there is anything specific the human would like to know about it. The human asks how AI can be used in healthcare. The AI explains that AI can be used in healthcare in a variety of ways, such as medical imaging analysis, predicting patient outcomes, drug discovery, and development, and providing patient support through chatbots and virtual assistants. AI has the potential to greatly improve healthcare outcomes and efficiency. Specifically, AI can be used in healthcare to analyze medical images and assist in diagnosing diseases, predict patient outcomes, aid i

In [20]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human greets the AI and asks how it is doing. The AI responds by returning the greeting and asking how the human is doing. The human then asks the AI to tell them about itself. The AI explains that it is an AI language model called GPT-3 developed by OpenAI. It has been trained on various data sources and can assist with tasks, answer questions, engage in conversations, and provide creative writing suggestions. The AI asks if there is anything specific the human would like to know about it. The human asks how AI can be used in healthcare. The AI explains that AI can be used in healthcare in a variety of ways, such as medical imaging analysis, predicting patient outcomes, drug discovery, and development, and providing patient support through chatbots and virtual assistants. AI has the potential to greatly improve healthcare outcomes and efficiency. Specifically, AI can be used in healthcare to analyze medical images and assist in diagnosing diseas